In [1]:
from model import LDM
import torch
import torch.nn as nn
from torch.distributions import Normal
import pandas as pd
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class FeatureMapper(nn.Module):
    def __init__(self, input_dim, embedding_dim, dropout = 0.1):
        super(FeatureMapper, self).__init__()
        self.input_dim = input_dim
        self.embedding_dim = embedding_dim

        self.feature_net = nn.Sequential(
            nn.Linear(self.input_dim, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(16, self.embedding_dim)
        )

    def forward(self, x):
        return self.feature_net(x)

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embedding_dim = 14
n_epochs = 100
Aij = torch.tensor([[0, 2, 0, 3, 1, 2, 0, 0, 2, 0, 1, 0], 
                    [0, 0, 2, 0, 1, 0, 3, 0, 0, 1, 0, 0],
                    [3, 3, 0, 0, 0, 1, 0, 3, 0, 0, 0, 1],
                    [3, 3, 0, 0, 0, 2, 0, 0, 1, 0, 1, 0],
                    [0, 0, 2, 0, 0, 0, 3, 0, 1, 0, 0, 0],
                    [1, 2, 0, 3, 1, 2, 0, 0, 2, 0, 1, 0], 
                    [0, 0, 2, 0, 1, 0, 0, 1, 0, 1, 0, 0],
                    [0, 3, 1, 0, 0, 1, 0, 3, 0, 0, 0, 1],
                    [3, 3, 0, 0, 0, 1, 0, 0, 0, 0, 1, 2],
                    [0, 0, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0]],dtype=torch.float32, device=device)
lr = 0.01
seed = 20
ldm_trained = LDM(Aij, embedding_dim, device, n_epochs, lr, seed)
ldm_trained.train()
Aij_probs_true = ldm_trained.probit()  # Compute the probit probability matrix
loss_out = ldm_trained.train()
w, v = ldm_trained.get_embeddings()

Epoch 0/100, Loss: 2.0754
Epoch 1/100, Loss: 2.0702
Epoch 2/100, Loss: 2.0649
Epoch 3/100, Loss: 2.0535
Epoch 4/100, Loss: 2.0504
Epoch 5/100, Loss: 2.0443
Epoch 6/100, Loss: 2.0389
Epoch 7/100, Loss: 2.0221
Epoch 8/100, Loss: 2.0044
Epoch 9/100, Loss: 1.9920
Epoch 10/100, Loss: 1.9766
Epoch 11/100, Loss: 1.9610
Epoch 12/100, Loss: 1.9419
Epoch 13/100, Loss: 1.9246
Epoch 14/100, Loss: 1.9094
Epoch 15/100, Loss: 1.8909
Epoch 16/100, Loss: 1.8653
Epoch 17/100, Loss: 1.8479
Epoch 18/100, Loss: 1.8231
Epoch 19/100, Loss: 1.8078
Epoch 20/100, Loss: 1.7846
Epoch 21/100, Loss: 1.7574
Epoch 22/100, Loss: 1.7359
Epoch 23/100, Loss: 1.7124
Epoch 24/100, Loss: 1.6820
Epoch 25/100, Loss: 1.6564
Epoch 26/100, Loss: 1.6261
Epoch 27/100, Loss: 1.5994
Epoch 28/100, Loss: 1.5734
Epoch 29/100, Loss: 1.5485
Epoch 30/100, Loss: 1.5256
Epoch 31/100, Loss: 1.5044
Epoch 32/100, Loss: 1.4850
Epoch 33/100, Loss: 1.4650
Epoch 34/100, Loss: 1.4491
Epoch 35/100, Loss: 1.4338
Epoch 36/100, Loss: 1.4197
Epoch 37/10

/Users/christine/LatentDistanceModel/latent_distance/model.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.freqs = torch.tensor(self.counts, dtype=torch.float32, device= self.device)


In [20]:
f_vec = torch.tensor([[0, 1, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 1, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 300, 0, 0, 0], 
                      [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 300, 0, 0, 0], 
                      [0, 0, 0, 0, 1, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0], 
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 40, 0], 
                      [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 0, 0],
                      [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 0, 0, 0, 0], 
                      [0, 0, 0, 1, 0, 0, 0, 0, 47, 0, 0, 0, 0, 0, 0], 
                      [0, 1, 0, 0, 0, 0, 34, 0, 0, 0, 0, 0, 0, 0, 0], 
                      [1, 0, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17]
                      ], dtype=torch.float32, device=device)
adj_matrix_idx = pd.read_csv('/Users/christine/LatentDistanceModel/data/adj_matrix_idx.csv', sep =',')
a_names = adj_matrix_idx[0:Aij.shape[0]]
a_idx = {drug_id: idx for idx, drug_id in enumerate(a_names)}
a_idx = np.array([a_idx[drug_id] for drug_id in a_names])
f_vec_idx = np.array([0,0 , 1,1,  2, 3, 4, 5, 6, 7, 8, 9])
print(f'Aij_idx: {a_idx}. There are {Aij.shape[0]} drugs in Aij and {len(a_idx)} drugs in the index.\nf_vec_idx: {f_vec_idx}. There are {f_vec.shape[0]} drugs in f_vec and {len(f_vec_idx)} drugs in the index.')

Aij_idx: [0]. There are 10 drugs in Aij and 1 drugs in the index.
f_vec_idx: [0 0 1 1 2 3 4 5 6 7 8 9]. There are 12 drugs in f_vec and 12 drugs in the index.


In [21]:
a_idx, f_vec_idx

(array([0]), array([0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [29]:
mapper = FeatureMapper(input_dim=f_vec.shape[1], embedding_dim=w.shape[1])
optimizer = torch.optim.Adam(mapper.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
num_epochs = 100

w_frozen = ldm_trained.w.detach().clone()
f_vec_tensor = torch.tensor(f_vec, dtype=torch.float32)
drug_idx_tensor = torch.tensor(f_vec_idx, dtype=torch.long)
w_tensor = torch.tensor(w, dtype=torch.float32)

for epoch in range(num_epochs):
    mapper.train()
    optimizer.zero_grad()

    z_pred = mapper(f_vec_tensor)
    z_true = w_tensor[drug_idx_tensor]
    loss = loss_fn(z_pred, z_true)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 1.4708
Epoch 1, Loss: 1.4316
Epoch 2, Loss: 1.1256
Epoch 3, Loss: 1.1074
Epoch 4, Loss: 1.0076
Epoch 5, Loss: 0.9700
Epoch 6, Loss: 1.0012
Epoch 7, Loss: 0.9267
Epoch 8, Loss: 0.9377
Epoch 9, Loss: 0.9270
Epoch 10, Loss: 0.9738
Epoch 11, Loss: 0.9413
Epoch 12, Loss: 0.8992
Epoch 13, Loss: 0.9235
Epoch 14, Loss: 0.9703
Epoch 15, Loss: 0.9127
Epoch 16, Loss: 0.8497
Epoch 17, Loss: 0.8636
Epoch 18, Loss: 0.8517
Epoch 19, Loss: 0.8328
Epoch 20, Loss: 0.7894
Epoch 21, Loss: 0.8503
Epoch 22, Loss: 0.8208
Epoch 23, Loss: 0.8155
Epoch 24, Loss: 0.7675
Epoch 25, Loss: 0.8518
Epoch 26, Loss: 0.9334
Epoch 27, Loss: 0.8464
Epoch 28, Loss: 0.8073
Epoch 29, Loss: 0.8324
Epoch 30, Loss: 0.7798
Epoch 31, Loss: 0.7914
Epoch 32, Loss: 0.7120
Epoch 33, Loss: 0.6943
Epoch 34, Loss: 0.7553
Epoch 35, Loss: 0.7029
Epoch 36, Loss: 0.7217
Epoch 37, Loss: 0.7539
Epoch 38, Loss: 0.6656
Epoch 39, Loss: 0.6935
Epoch 40, Loss: 0.8399
Epoch 41, Loss: 0.7595
Epoch 42, Loss: 0.6322
Epoch 43, Loss: 0.645

/var/folders/nd/td49stzx1sb1x3vskq7xg2540000gn/T/ipykernel_69771/3297555768.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  f_vec_tensor = torch.tensor(f_vec, dtype=torch.float32)
/var/folders/nd/td49stzx1sb1x3vskq7xg2540000gn/T/ipykernel_69771/3297555768.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w_tensor = torch.tensor(w, dtype=torch.float32)


In [63]:
w_tensor[drug_idx_tensor].shape

torch.Size([12, 14])

# Real data

In [3]:
def load_data(path_to_csv, device):
    df = pd.read_csv(path_to_csv, index_col=0)
    Aij = torch.tensor(df.values, dtype=torch.float32).to(device)
    return Aij

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
csv_path = "/Users/christine/LatentDistanceModel/data/filtered_adj_matrix.csv" 
Aij_real = load_data(csv_path, device)
print(Aij_real.shape)

#importing the data
feature_vector = pd.read_csv('/Users/christine/LatentDistanceModel/data/feature_vector.tsv', sep='\t')
feature_vector.drop(columns=['Unnamed: 0'], inplace=True)
feature_vector

torch.Size([745, 3677])


,0,Chewing gum,Inhal,Inhal.aerosol,Inhal.powder,Inhal.solution,N,O,P,R,...,V08AB05,V08AB06,V08AB07,V08AB09,V08CA03,V08CA04,V08CA06,V08CA08,V08CA09,V09AB03
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
#find indexing for the two vectors
adj_matrix_names = pd.read_csv('/Users/christine/LatentDistanceModel/data/filtered_adj_matrix.csv', sep=',')
adj_matrix_idx = adj_matrix_names['Stitch flat']
adj_matrix_idx.to_csv('/Users/christine/LatentDistanceModel/data/adj_matrix_idx.csv', index=False)
adj_matrix_idx = adj_matrix_idx.to_numpy()

#index of feature vector
feature_vector_names = pd.read_csv('/Users/christine/LatentDistanceModel/data/feature_vector_names.tsv', sep = '\t', )
feature_vector_names['ID Adm.Rs'] = feature_vector_names['ID Adm.Rs'].str.split('_').str[0]
feature_vector_idx = feature_vector_names['ID Adm.Rs'].to_numpy()
adj_matrix_idx, feature_vector_idx

(array(['CID100000085', 'CID100000137', 'CID100000143', 'CID100000158',
        'CID100000159', 'CID100000160', 'CID100000191', 'CID100000214',
        'CID100000232', 'CID100000247', 'CID100000271', 'CID100000311',
        'CID100000444', 'CID100000450', 'CID100000453', 'CID100000581',
        'CID100000596', 'CID100000598', 'CID100000727', 'CID100000738',
        'CID100000750', 'CID100000772', 'CID100000861', 'CID100000937',
        'CID100000942', 'CID100001065', 'CID100001125', 'CID100001134',
        'CID100001301', 'CID100001546', 'CID100001690', 'CID100001775',
        'CID100001805', 'CID100001971', 'CID100001972', 'CID100001978',
        'CID100001983', 'CID100001990', 'CID100002019', 'CID100002022',
        'CID100002082', 'CID100002083', 'CID100002088', 'CID100002092',
        'CID100002094', 'CID100002099', 'CID100002118', 'CID100002130',
        'CID100002153', 'CID100002156', 'CID100002159', 'CID100002160',
        'CID100002161', 'CID100002162', 'CID100002170', 'CID1000

In [25]:
#for real data
Aij_dic = {drug_id: idx for idx, drug_id in enumerate(adj_matrix_idx)}
unique_drugs_Aij = pd.unique(adj_matrix_idx)
Aij_idx = np.array([Aij_dic[drug_id] for drug_id in adj_matrix_names['Stitch flat']])

#for feature vector
unique_drugs_f = pd.unique(feature_vector_idx)
f_dic = {drug_id: idx for idx, drug_id in enumerate(unique_drugs_f)}
f_idx = np.array([f_dic[drug_id] for drug_id in feature_vector_names['ID Adm.Rs']])
f_idx

array([  0,   0,   1, ..., 742, 743, 744], shape=(1095,))

In [6]:
only_in_Aij = set(unique_drugs_Aij) - set(unique_drugs_f)
only_in_f = set(unique_drugs_f) - set(unique_drugs_Aij)
not_in_both = only_in_Aij.union(only_in_f)
print(f"Only in Aij: {only_in_Aij}\nOnly in feature vector: {only_in_f}\nNot in either: {not_in_both}")
print(f"{len(only_in_Aij)} are missing from feature vector")

Only in Aij: set()
Only in feature vector: set()
Not in either: set()
0 are missing from feature vector


In [7]:
#convert to tensor
feature_tensor = torch.tensor(feature_vector.astype(np.float32).to_numpy(), dtype=torch.float32)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embedding_dim = 15
lr = 0.005
seed = 20
n_epochs = 1000
ldm_trained_r = LDM(Aij_real, embedding_dim, device, n_epochs, lr, seed)
ldm_trained_r.train()
Aij_probs_true_r = ldm_trained_r.probit()  # Compute the probit probability matrix
loss_out_r = ldm_trained_r.train()
w_r, v_r = ldm_trained_r.get_embeddings()


/Users/christine/LatentDistanceModel/latent_distance/model.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.freqs = torch.tensor(self.counts, dtype=torch.float32, device= self.device)


Epoch 0/1000, Loss: 0.1004
Epoch 1/1000, Loss: 0.0994
Epoch 2/1000, Loss: 0.0985
Epoch 3/1000, Loss: 0.0978
Epoch 4/1000, Loss: 0.0971
Epoch 5/1000, Loss: 0.0966
Epoch 6/1000, Loss: 0.0961
Epoch 7/1000, Loss: 0.0957
Epoch 8/1000, Loss: 0.0954
Epoch 9/1000, Loss: 0.0951
Epoch 10/1000, Loss: 0.0947
Epoch 11/1000, Loss: 0.0944
Epoch 12/1000, Loss: 0.0940
Epoch 13/1000, Loss: 0.0937
Epoch 14/1000, Loss: 0.0933
Epoch 15/1000, Loss: 0.0930
Epoch 16/1000, Loss: 0.0926
Epoch 17/1000, Loss: 0.0922
Epoch 18/1000, Loss: 0.0916
Epoch 19/1000, Loss: 0.0909
Epoch 20/1000, Loss: 0.0904
Epoch 21/1000, Loss: 0.0901
Epoch 22/1000, Loss: 0.0896
Epoch 23/1000, Loss: 0.0893
Epoch 24/1000, Loss: 0.0892
Epoch 25/1000, Loss: 0.0891
Epoch 26/1000, Loss: 0.0888
Epoch 27/1000, Loss: 0.0886
Epoch 28/1000, Loss: 0.0885
Epoch 29/1000, Loss: 0.0884
Epoch 30/1000, Loss: 0.0882
Epoch 31/1000, Loss: 0.0880
Epoch 32/1000, Loss: 0.0879
Epoch 33/1000, Loss: 0.0878
Epoch 34/1000, Loss: 0.0875
Epoch 35/1000, Loss: 0.0873
Ep

In [9]:
mapper = FeatureMapper(input_dim=feature_vector.shape[1], embedding_dim=w_r.shape[1])
optimizer = torch.optim.Adam(mapper.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
num_epochs = 100

w_frozen = ldm_trained_r.w.detach().clone()
feature_vec_tensor = torch.tensor(feature_vector.astype(np.float32).to_numpy(), dtype=torch.float32)
feature_idx_tensor = torch.tensor(f_idx, dtype=torch.long)
w_tensor = torch.tensor(w_r, dtype=torch.float32)

for epoch in range(num_epochs):
    mapper.train()
    optimizer.zero_grad()

    z_pred = mapper(feature_vec_tensor)
    z_true = w_tensor[feature_idx_tensor]
    loss = loss_fn(z_pred, z_true)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 5.5041
Epoch 1, Loss: 5.1588
Epoch 2, Loss: 4.0435
Epoch 3, Loss: 3.5230
Epoch 4, Loss: 3.0633
Epoch 5, Loss: 2.9870
Epoch 6, Loss: 2.9334
Epoch 7, Loss: 2.7287
Epoch 8, Loss: 2.6391
Epoch 9, Loss: 2.6156
Epoch 10, Loss: 2.5718
Epoch 11, Loss: 2.5182
Epoch 12, Loss: 2.6057
Epoch 13, Loss: 2.4620
Epoch 14, Loss: 2.4548
Epoch 15, Loss: 2.4529
Epoch 16, Loss: 2.3815
Epoch 17, Loss: 2.4082
Epoch 18, Loss: 2.3822
Epoch 19, Loss: 2.3460
Epoch 20, Loss: 2.3473
Epoch 21, Loss: 2.3767
Epoch 22, Loss: 2.3656
Epoch 23, Loss: 2.3422
Epoch 24, Loss: 2.3361
Epoch 25, Loss: 2.3318
Epoch 26, Loss: 2.3214
Epoch 27, Loss: 2.2938
Epoch 28, Loss: 2.3153
Epoch 29, Loss: 2.2852
Epoch 30, Loss: 2.2909
Epoch 31, Loss: 2.2999
Epoch 32, Loss: 2.2907
Epoch 33, Loss: 2.2703
Epoch 34, Loss: 2.2662
Epoch 35, Loss: 2.2618
Epoch 36, Loss: 2.2680
Epoch 37, Loss: 2.2620
Epoch 38, Loss: 2.2589
Epoch 39, Loss: 2.2365
Epoch 40, Loss: 2.2380
Epoch 41, Loss: 2.2502
Epoch 42, Loss: 2.2259
Epoch 43, Loss: 2.223

/var/folders/nd/td49stzx1sb1x3vskq7xg2540000gn/T/ipykernel_66147/2579014675.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w_tensor = torch.tensor(w_r, dtype=torch.float32)


# End to end

In [10]:
f_vector = pd.read_csv('/Users/christine/LatentDistanceModel/data/feature_vector_combined.csv', sep=',')
f_vector.drop(columns=['base_id'], inplace=True)

In [23]:
class EndToEnd(nn.Module):
    def __init__(self, feature_mapper, v, gamma, beta, beta_thilde, a, b, Aij, device):
        super().__init__()
        self.feature_mapper = feature_mapper
        self.v = nn.Parameter(v.clone())
        self.gamma = nn.Parameter(gamma.clone()) 
        self.beta = nn.Parameter(beta.clone())  
        self.beta_thilde = nn.Parameter(beta_thilde.clone())  
        self.a = nn.Parameter(a.clone())
        self.b = nn.Parameter(b.clone())

        self.Aij = Aij
        self.device = device
        self.n_ordinal_classes = Aij.max().int().item() + 1
        self.n_drugs, self.n_effects = Aij.shape

    def get_thresholds(self):
        # reused from LDM
        deltas = torch.softmax(self.beta_thilde, dim = 0)  
        thresholds = torch.cumsum(deltas, dim=0)* self.a - self.b
        return torch.cat([torch.tensor([-float("inf")], device=self.device), thresholds, torch.tensor([float("inf")], device=self.device)])
    
    def forward(self, feature_vector, Aij_idx):
        w_pred = self.feature_mapper(feature_vector) #now what is being modelled is the features from the feature vec
        Aij_idxed = self.Aij[Aij_idx]
        normal_dist = Normal(0, 1) 
        probit_matrix = torch.zeros((self.n_ordinal_classes, self.n_drugs, self.n_effects), device=self.device)
        thresholds = self.get_thresholds()
    
        #Linear term (\beta^T x_{i,j})
        linear_term = torch.matmul(Aij_idxed, self.beta.unsqueeze(1))

        # Distance term -|w_i - v_j|
        dist = -torch.norm(w_pred.unsqueeze(1) - self.v.unsqueeze(0), dim=2)

        # Latent variable \beta^T x_{i,j} + \alpha(u_i - u_j)
        latent_var = self.gamma + linear_term + dist
        
        for y in range(self.n_ordinal_classes):
            z1 = latent_var - thresholds[y]
            z2 = latent_var - thresholds[y+1]
            probit_matrix[y, :, :] = normal_dist.cdf(z1) - normal_dist.cdf(z2)
        return probit_matrix
    
    def ordinal_cross_entropy_loss(self, probit_matrix):
    # Compute the predicted probabilities using the probit function

        # Initialize loss variable
        loss = 0.0

        # Convert Aij to a one-hot encoded tensor
        one_hot_target = torch.zeros(self.n_drugs, self.n_effects, self.n_ordinal_classes, device=self.device)
        one_hot_target.scatter_(-1, self.Aij.unsqueeze(-1).long(), 1)  # One-hot encoding

        # Compute the log-likelihood loss efficiently
        prob = probit_matrix  # Shape: (n_ordinal_classes, n_drugs, n_effects)
        loss = -torch.mean(torch.log(torch.sum(prob * one_hot_target.permute(2, 0, 1), dim=0) + 1e-8))
        return loss
    
    def _get_learned_params(self):
        return self.v, self.gamma, self.beta, self.beta_thilde, self.a, self.b

In [32]:
beta = ldm_trained.beta.detach().cpu()
beta.shape

torch.Size([12])

In [34]:
v = ldm_trained.v.detach().cpu()
v.shape

torch.Size([12, 14])

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Aij_tensor = torch.tensor(Aij_real, dtype=torch.float32, device=device)
feat_vec = feature_tensor[f_idx]
f_vec_tensor = torch.tensor(f_vector.values, dtype=torch.float32, device=device)

v = ldm_trained_r.v.detach().cpu()
gamma = ldm_trained_r.gamma.detach().cpu()
beta = ldm_trained_r.beta.detach().cpu()
beta_thilde = ldm_trained_r.beta_thilde.detach().cpu()
a = ldm_trained_r.a.detach().cpu()
b = ldm_trained_r.b.detach().cpu()

model = EndToEnd(
    feature_mapper=FeatureMapper(feature_tensor.shape[1], embedding_dim=w_r.shape[1]).to(device),
    v=ldm_trained_r.v,
    gamma=ldm_trained_r.gamma,
    beta=ldm_trained_r.beta,
    beta_thilde=ldm_trained_r.beta_thilde,
    a=ldm_trained_r.a,
    b=ldm_trained_r.b,
    Aij=Aij_tensor,
    device=device
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training
for epoch in range(20):
    model.train()
    optimizer.zero_grad()

    probs = model(f_vec_tensor, Aij_idx)
    loss = model.ordinal_cross_entropy_loss(probs)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


/var/folders/nd/td49stzx1sb1x3vskq7xg2540000gn/T/ipykernel_66147/2241559983.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Aij_tensor = torch.tensor(Aij_real, dtype=torch.float32, device=device)


Epoch 1, Loss: 2.2309
Epoch 2, Loss: 1.5778
Epoch 3, Loss: 1.3697
Epoch 4, Loss: 1.2007
Epoch 5, Loss: 1.0433
Epoch 6, Loss: 0.9013
Epoch 7, Loss: 0.8303
Epoch 8, Loss: 0.7747
Epoch 9, Loss: 0.7252
Epoch 10, Loss: 0.6694
Epoch 11, Loss: 0.6082
Epoch 12, Loss: 0.5770
Epoch 13, Loss: 0.5392
Epoch 14, Loss: 0.4589
Epoch 15, Loss: 0.3953
Epoch 16, Loss: 0.3665
Epoch 17, Loss: 0.3599
Epoch 18, Loss: 0.3598
Epoch 19, Loss: 0.3600
Epoch 20, Loss: 0.3600


# Prediction

In [27]:
v, gamma, beta, beta_thilde, a, b = model._get_learned_params()
v

Parameter containing:
tensor([[-1.6013,  3.1512, -2.8779,  ..., -5.0465, -2.5489,  1.5419],
        [-2.3222, -0.3113,  3.0252,  ...,  3.0597, -1.9720,  0.4039],
        [-4.6603,  2.5253,  3.2874,  ...,  1.3841, -3.9984,  3.1362],
        ...,
        [ 0.3501,  3.8984,  2.6303,  ...,  2.7051,  3.1733,  1.1744],
        [ 3.5881,  0.3375,  3.9988,  ...,  3.2680, -4.9407, -2.6678],
        [-3.7470,  3.9682,  3.3612,  ...,  2.8748, -3.6547,  3.2776]],
       requires_grad=True)

In [30]:
new_drug = f_vector.iloc[0,:]
new_drug.iloc[4] = 1
new_drug_tensor = torch.tensor(new_drug.values, dtype=torch.float32, device=device).unsqueeze(0)
new_drug_tensor.shape

torch.Size([1, 1011])

In [28]:
model.eval()
with torch.no_grad():
    drug_id = 0  # Dummy drug id
    effect_ids = torch.arange(model.n_effects, dtype=torch.long, device=device)
    Aij_idxs = drug_id * model.n_effects + effect_ids
    probit_matrix = model(new_drug_tensor, Aij_idxs)
    predicted_classes = torch.argmax(probit_matrix.squeeze(1), dim=0)

IndexError: index 1838 is out of bounds for dimension 0 with size 745

In [22]:
model.eval()
with torch.no_grad():
    # 1. Map drug feature vector to latent space
    new_drug_tensor = torch.tensor(new_drug.values, dtype=torch.float32, device=device).unsqueeze(0)
    w_pred = model.feature_mapper(new_drug_tensor)  # shape: (1, D)

    # 2. Use w_pred and side effect embeddings (v) to compute distances/probits
    # Assuming model.v is of shape (n_effects, D)
    # Repeat w_pred across all effects
    w_repeated = w_pred.repeat(model.n_effects, 1)  # shape: (n_effects, D)
    v_effects = model.v 
        # 3. Compute squared Euclidean distances or other relevant distance metric
    distances = torch.norm(w_repeated - v_effects, dim=1)  # shape: (n_effects,)

    # 4. Convert distances to logits via the latent distance model (e.g., probit)
    normal_dist = torch.distributions.Normal(0, 1)
    probits = 1 - normal_dist.cdf(distances)

    # 5. (Optional) Apply a threshold or argmax to make class predictions
    predicted_labels = (probits > 0.5).int()  # Binary thresholding, for example
predicted_labels

tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int32)

In [32]:
Aij_real.shape

torch.Size([745, 3677])

In [33]:
import torch

# 1. Set model to eval mode
model.eval()

# 2. Prepare the new drug feature vector
# Assuming new_drug_features is a 1D numpy array or pandas Series
new_drug_tensor = torch.tensor(new_drug, dtype=torch.float32).to(device)
new_drug_tensor = new_drug_tensor.unsqueeze(0)  # Add batch dimension if needed

Aij_idx = torch.stack([
    torch.zeros(Aij_real.shape[1], dtype=torch.long, device=device),  # dummy drug idx
    torch.arange(Aij_real.shape[1], dtype=torch.long, device=device)  # effect idx
], dim=1)
# 3. Predict using the model
with torch.no_grad():

    # Predict probit scores (shape: [n_classes, 1, n_effects])
    probit_output = model(new_drug_tensor, Aij_idx)

    # Reduce output to shape: (n_effects,)
    predicted_classes = torch.argmax(probit_output.squeeze(1), dim=0)


/var/folders/nd/td49stzx1sb1x3vskq7xg2540000gn/T/ipykernel_66147/2923591087.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_drug_tensor = torch.tensor(new_drug, dtype=torch.float32).to(device)


IndexError: index 2757 is out of bounds for dimension 0 with size 745